In [1]:
import sys
sys.path.append('/users/gxb18167/EEG-NLP')

from src import data
from src import utils
from src import Networks

import numpy as np
import torch
from torch.utils.data import DataLoader, TensorDataset
import torch.nn as nn
import torch.optim as optim

C:\Users\gxb18167\Anaconda3\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\gxb18167\Anaconda3\lib\site-packages\numpy\.libs\libopenblas64__v0.3.21-gcc_10_3_0.dll
C:\Users\gxb18167\Anaconda3\lib\site-packages\numpy\.libs\libopenblas64__v0.3.23-gcc_10_3_0.dll
  warnings.warn("loaded more than 1 DLL from .libs:"
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\gxb18167\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
train_path = r"C:\Users\gxb18167\PycharmProjects\SIGIR_EEG_GAN\Development\Named-Entity-Classification\Data-Management\train_NER.pkl"

test_path = r"C:\Users\gxb18167\PycharmProjects\SIGIR_EEG_GAN\Development\Named-Entity-Classification\Data-Management\test_NER.pkl"

In [3]:
d = data.Data()
util = utils.Utils()

In [4]:
train_NE, train_EEG_segments, train_Classes = d.NER_read_custom_files(train_path)

In [5]:
train_NE[0]

['Henry', 'Ford']

In [6]:
vector_size = 50
#parameters['vector_size'] = vector_size
window = 5
#parameters['window'] = window
min_count = 1
#parameters['min_count'] = min_count
workers = 4

In [7]:
train_word_embeddings, train_NE_embedded = util.NER_Word2Vec(train_NE, vector_size, window, min_count, workers)

In [8]:
len(train_NE_embedded)

585

In [14]:
train_NE_expanded = util.NER_expanded_NER_list(train_EEG_segments, train_NE_embedded)

In [15]:
max_seq_len = max([len(i) for i in train_NE_expanded])

In [17]:
#padding function
for i in range(len(train_NE_expanded)):
    padding_count = max_seq_len - len(train_NE_expanded[i])
    for j in range(padding_count):





AttributeError: 'numpy.ndarray' object has no attribute 'append'